In [11]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [3]:
def preprocessing(img):
    alpha = 1.5
    beta = 0.5
    # adjusted = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # resize = cv2.resize(gray, (64, 64))
    blurred = cv2.GaussianBlur(gray, (5,5), 0)

    thres = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    return thres

In [4]:
def featureExtraction(img):
    mean, var = cv2.meanStdDev(img)

    skewness = np.mean((img - mean) ** 3) / var ** 1.5
    kurtosis = np.mean((img - mean) ** 4) / var ** 2 - 3

    moments = cv2.moments(img)

    hu_moments = cv2.HuMoments(moments)
    hu_moments = -np.sign(hu_moments) * np.log10(np.abs(hu_moments))

    features = np.concatenate((mean, var, skewness, kurtosis, hu_moments), axis=None)
    return features

In [23]:
dataset_dir = 'dataset'
class_names = ['real', 'fake']

data = []
labels = []

for class_name in class_names:
    class_dir = os.path.join(dataset_dir, class_name)

    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        image = cv2.imread(image_path)

        preprocessed_images = []

        preprocessed = preprocessing(image)

        moments = featureExtraction(preprocessed)

        data.append(moments)
        labels.append(class_names.index(class_name))
        

In [8]:
data = np.array(data)
labels = np.array(labels)

X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size=0.3, random_state=42)

clf = SVC(kernel='linear', C=1.0, random_state=42)
clf.fit(X_train, Y_train)

Y_pred = clf.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred)

print(f'Accuracy: {accuracy: .2f}, Precision: {precision: .2f}, Recall: {recall: .2f}, F1-score: {f1: .2f}')

Accuracy:  0.77, Precision:  0.77, Recall:  0.74, F1-score:  0.76


In [24]:
path = 'real_test.jpg'

test_image = cv2.imread(path)

# plt.imshow(test_image)

test_preprocessed = preprocessing(test_image)

test_features = featureExtraction(test_preprocessed)

prediction = clf.predict([test_features])[0]

if prediction == 0:
    print('real')

else:
    print('fake')

real
